# Table of Contents

# Introduction

## Overview

This notebook is a draft of the in-progress reproduction of the paper "Multi-modal Molecule Structure-text Model for Text-based Retrieval and Editing" originally authored by Liu et al. (2023).

In this section we provide background on the purpose of this research, as well as an overview of the approach concerning structural and textual representation of molecules, as well as the criticality of pretraining for the MoleculeSTM modal.

## Background

Drug discovery is a complex, time-consuming, and costly process that involves identifying molecules with therapeutic potential. To that point, recent progress in artificial intelligence (AI) offers a way to minimize these aspects and transform the process as a whole. 

The conventional approach for AI-facilitated drug discovery relies heavily on understanding the chemical structures of molecules, often overlooking the vast, unstructured textual knowledge available, which could offer insights into new drug design objectives, adaptability to text-based instructions, and predictions of complex biological activities. Indeed, Liu et al. (2023) breathe life into this notion with their introduction of MoleculeSTM, a multi-modal molecule structure-text modal that leverages both molecular structural information and textual descriptions through a contrastive learning strategy. 


### Structural and Textual Representation of Molecules

MoleculeSTM represents a multi-modal foundation model that concurrently learns from molecules' chemical structures and their associated textual descriptions. This dual-branch approach, incorporating a chemical structure branch and a textual description branch, allows for the integration of existing molecular structural models and scientific language models (Liu et al., 2023). The contrastive learning paradigm bridges these two branches, facilitating a comprehensive understanding of molecular data.

### Criticality of Pretraining

To enable training for the model given the two distinct branches described, the authors have synthesized this data into a structure-text dataset; in which each chemical structure is associated with a textual description. The open-ended format of the textual descriptions allow the model to achieve better zero-shot performance i.e. generalize to unseen data (Liu et al., 2023). Using the PubChemSTM dataset, the authors pretrain the model via contrastive learning. The general framework is to map representations from the structural and textual branches to a shared molecular model by reducing the distance between pairs of the same molecule and decreasing that between pairs for different molecules. 

## Scope of Reproducability

### A Note On This Notebook

Given the complexity of this project, it is unfortunately a very prohibitive process to migrate the code, as well as the environmental dependencies, into a Google Colab notebook. To the extent that this draft is intended to reflect progress made on this project, we will save time attempting to make all of this code fit into that format in favor on making real progress towards the end goal of reproducing results. 

### Hypotheses

Here we list out the hypotheses from the original paper that we will be testing for our reproduction.

#### Zero-shot structure-text retrieval

MoleculeSTM can accurately link molecular structures to their textual descriptions without being directly trained on these specific pairs, showcasing its ability to generalize from its pretraining.

#### Zero-shot text-based molecule editing

MoleculeSTM can edit and generate molecular structures to meet new specifications provided via textual descriptions, demonstrating its understanding of complex chemical properties and functionalities from text alone.

#### Molecular property prediction

MoleculeSTM can predict molecular properties accurately, leveraging its pretraining on structure-text relationships, suggesting that it captures meaningful chemical information that is relevant for property prediction tasks.

# Setup

# Methodology

## Data

Due to the massive size of the data, as well as the need to keep the various datasets aligned with each other for the purposes of pretraining, we are unable to push the datasets with the limitatioms of pushes to a GitHub repository, nor can we simply upload a sample of the data. However, we will provide the link to download the data from HuggingFace [here](https://huggingface.co/datasets/chao1224/MoleculeSTM), similarly to how we accessed the data for ourselves. In addition, we provide this script adapted from the original code to download all the required training and downstream datasets that will be used for evaluation:

In [ ]:
from huggingface_hub import HfApi, snapshot_download
api = HfApi()
snapshot_download(repo_id="chao1224/MoleculeSTM", repo_type="dataset", local_dir='.')

In addition to this, there is a required step to perform preprocessing on the PubChemSTM dataset; this is due to the fact that the PubChemSTM dataset only has the license to share the structural representations of molecules, but not the textual represetntation. So, in order to complete the dataset required for pretraining the model, you must follow the instructions below adapted from the advice of Shangchao Liu:

### Step 1: Description Extraction

In [ ]:
import requests
from tqdm import tqdm
from collections import defaultdict
import json


def clean_up_description(description):
    description = description + " "

    ##### extra adj Pure #####
    if description.startswith("Pure "):
        description = description.replace("Pure ", "")
    ##### fix typo #####
    if description.startswith("Mercurycombines"):
        description = description.replace("Mercurycombines", "Mercury combines")
    
    name_special_case_list = [
        '17-Hydroxy-6-methylpregna-3,6-diene-3,20-dione. ',
        '5-Thymidylic acid. ',
        "5'-S-(3-Amino-3-carboxypropyl)-5'-thioadenosine. ",
        "Guanosine 5'-(trihydrogen diphosphate), monoanhydride with phosphorothioic acid. ",
        "5'-Uridylic acid. ",
        "5'-Adenylic acid, ",
        "Uridine 5'-(tetrahydrogen triphosphate). ",
        "Inosine 5'-Monophosphate. ",
        "Pivaloyloxymethyl butyrate (AN-9), ",
        "4-Amino-5-cyano-7-(D-ribofuranosyl)-7H- pyrrolo(2,3-d)pyrimidine. ",
        "Cardamonin (also known as Dihydroxymethoxychalcone), ",
    ]

    ##### a special case #####
    description = description.replace("17-Hydroxy-6-methylpregna-3,6-diene-3,20-dione. ", "17-Hydroxy-6-methylpregna-3,6-diene-3,20-dione is ")

    ##### a special case #####
    description = description.replace("5-Thymidylic acid. ", "5-Thymidylic acid. is ")

    ##### a special case #####
    description = description.replace("5'-S-(3-Amino-3-carboxypropyl)-5'-thioadenosine. ", "5'-S-(3-Amino-3-carboxypropyl)-5'-thioadenosine. is ")

    ##### a special case #####
    description = description.replace("Guanosine 5'-(trihydrogen diphosphate), monoanhydride with phosphorothioic acid. ", "Guanosine 5'-(trihydrogen diphosphate), monoanhydride with phosphorothioic acid is ")

    ##### a special case #####
    description = description.replace("5'-Uridylic acid. ", "5'-Uridylic acid is ")

    ##### a special case #####
    description = description.replace("5'-Adenylic acid, ", "5'-Adenylic acid is ")

    ##### a special case #####
    description = description.replace("Uridine 5'-(tetrahydrogen triphosphate). ", "Uridine 5'-(tetrahydrogen triphosphate). is ")

    ##### a special case #####
    description = description.replace("Inosine 5'-Monophosphate. ", "Inosine 5'-Monophosphate. is ")

    ##### a special case #####
    description = description.replace("Pivaloyloxymethyl butyrate (AN-9), ", "Pivaloyloxymethyl butyrate (AN-9) is ")

    ##### a special case #####
    description = description.replace("4-Amino-5-cyano-7-(D-ribofuranosyl)-7H- pyrrolo(2,3-d)pyrimidine. ", "4-Amino-5-cyano-7-(D-ribofuranosyl)-7H- pyrrolo(2,3-d)pyrimidine is ")

    ##### a special case #####
    description = description.replace("Cardamonin (also known as Dihydroxymethoxychalcone), ", "Cardamonin (also known as Dihydroxymethoxychalcone) is ")

    ##### a special case #####
    description = description.replace("Lithium has been used to treat ", "Lithium is ")

    ##### a special case #####
    description = description.replace("4,4'-Methylenebis ", "4,4'-Methylenebis is ")

    ##### a special case #####
    description = description.replace("2,3,7,8-Tetrachlorodibenzo-p-dioxin", "2,3,7,8-Tetrachlorodibenzo-p-dioxin is ")

    ##### a special case #####
    description = description.replace("Exposure to 2,4,5-trichlorophenol ", "2,4,5-Trichlorophenol exposure ")

    index = 0
    L = len(description)
    if description.startswith('C.I. '):
        start_index = len('C.I. ')
    elif description.startswith('Nectriapyrone. D '):
        start_index = len('Nectriapyrone. D ')
    elif description.startswith('Salmonella enterica sv. Minnesota LPS core oligosaccharide'):
        start_index = len('Salmonella enterica sv. Minnesota LPS core oligosaccharide')
    else:
        start_index = 0
    for index in range(start_index, L - 1):
        if index < L-2:
            if description[index] == '.' and description[index+1] == ' ' and 'A' <= description[index+2] <= 'Z':
                break
        elif index == L - 2:
            break
    
    first_sentence = description[:index+1]
    return first_sentence


def extract_name(name_raw, description):
    first_sentence = clean_up_description(description)

    splitter = '  --  --  '
    if ' are ' in first_sentence or ' were ' in first_sentence:
        replaced_words = 'These molecules'
    else:
        replaced_words = 'This molecule'

    first_sentence = first_sentence.replace(' is ', splitter)
    first_sentence = first_sentence.replace(' are ', splitter)
    first_sentence = first_sentence.replace(' was ', splitter)
    first_sentence = first_sentence.replace(' were ', splitter)
    first_sentence = first_sentence.replace(' appears ', splitter)
    first_sentence = first_sentence.replace(' occurs ', splitter)
    first_sentence = first_sentence.replace(' stands for ', splitter)
    first_sentence = first_sentence.replace(' belongs to ', splitter)
    first_sentence = first_sentence.replace(' exists ', splitter) # only for CID=11443
    first_sentence = first_sentence.replace(' has been used in trials ', splitter)
    first_sentence = first_sentence.replace(' has been investigated ', splitter)
    first_sentence = first_sentence.replace(' has many uses ', splitter)
    
    if splitter in first_sentence:
        extracted_name = first_sentence.split(splitter, 1)[0]
    elif first_sentence.startswith(name_raw):
        extracted_name = name_raw
    elif name_raw in first_sentence:
        extracted_name = name_raw
        extracted_name = None
        print("=====", name_raw)
        print("first sentence: ", first_sentence)
        # print()
    else:
        extracted_name = None

    if extracted_name is not None:
        extracted_description = description.replace(extracted_name, replaced_words)
    else:
        extracted_description = description

    return extracted_name, extracted_description, first_sentence


if __name__ == "__main__":
    total_page_num = 290
    # Please put your own dataset path here
    datasets_home_folder = "../../../Datasets"

    PubChemSTM_datasets_description_home_folder = "{}/step_01_PubChemSTM_description".format(datasets_home_folder)
    valid_CID_list = set()
    CID2name_raw, CID2name_extracted = defaultdict(list), defaultdict(list)
    CID2text_raw, CID2text_extracted = defaultdict(list), defaultdict(list)

    for page_index in tqdm(range(total_page_num)):
        page_num = page_index + 1
        compound_description_file_name = "Compound_description_{}.txt".format(page_num)
        f_out = open("{}/{}".format(PubChemSTM_datasets_description_home_folder, compound_description_file_name), "w")
        
        description_url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/annotations/heading/json?heading_type=Compound&heading=Record+Description&page={}".format(page_num)
        description_data = requests.get(description_url).json()

        description_data = description_data["Annotations"]
        assert description_data["Page"] == page_num
        assert description_data["TotalPages"] == total_page_num
        
        record_list = description_data["Annotation"]
        
        for record in record_list:
            try:
                CID = record["LinkedRecords"]["CID"][0]
                if "Name" in record:
                    name_raw = record["Name"]
                    CID2name_raw[CID].append(name_raw)
                else:
                    name_raw = None

                data_list = record["Data"]
                for data in data_list:
                    description = data["Value"]["StringWithMarkup"][0]["String"].strip()
                    
                    extracted_name, extracted_description, first_sentence = extract_name(name_raw, description)
                    if extracted_name is not None:
                        CID2name_extracted[CID].append(extracted_name)

                    CID_special_case_list = [45266824, 11683, 3759, 9700, 439155, 135398675, 135563708, 6030, 10238, 6133, 135398640, 77918, 60748, 11824, 641785, 11125, 7543, 15625, 7271]

                    ##### only for debugging #####
                    if CID in CID_special_case_list:
                        print("page: {}\tCID: {}".format(page_index, CID))                
                        if "Name" in record:
                            print('yes-name')
                            name = record["Name"]
                            print('name:', name)
                        else:
                            print('no-name')
                        print('extracted name:', extracted_name)
                        print("first_sentence:", first_sentence)
                        print("extracted_description:", extracted_description)
                        print("description:", description)
                        print() 
                        
                    CID2text_raw[CID].append(description)
                    CID2text_extracted[CID].append(extracted_description)

                    valid_CID_list.add(CID)
                    f_out.write("{}\n".format(CID))
                    f_out.write("{}\n\n".format(extracted_description))
            except:
                # print("===\n", record)
                # print("missing page: {}\tSourceName: {}\tSourceID: {}".format(page_index, record['SourceName'], record['SourceID']))
                continue
            
    valid_CID_list = list(set(valid_CID_list))
    valid_CID_list = sorted(valid_CID_list)
    # print("valid CID list: {}".format(valid_CID_list))
    print("Total CID (with raw name) {}".format(len(CID2name_raw)))
    print("Total CID (with extracted name) {}".format(len(CID2name_extracted)))
    print("Total CID {}".format(len(valid_CID_list)))
    
    with open("{}/PubChemSTM_data/raw/CID2name_raw.json".format(datasets_home_folder), "w") as f:
        json.dump(CID2name_raw, f)
    
    with open("{}/PubChemSTM_data/raw/CID2name.json".format(datasets_home_folder), "w") as f:
        json.dump(CID2name_extracted, f)

    with open("{}/PubChemSTM_data/raw/CID2text_raw.json".format(datasets_home_folder), "w") as f:
        json.dump(CID2text_raw, f)

    with open("{}/PubChemSTM_data/raw/CID2text.json".format(datasets_home_folder), "w") as f:
        json.dump(CID2text_extracted, f)

### Step 2: Download SDF

In [ ]:
import argparse
from PubChem_utils import download_and_extract_compound_file


parser = argparse.ArgumentParser()
parser.add_argument("--block_id", type=int, default=0)
args = parser.parse_args()


if __name__ == "__main__":
    datasets_home_folder = "../../../Datasets"

    PubChemSTM_datasets_home_folder = "{}/step_02_PubChemSTM_SDF".format(datasets_home_folder)
    block_id = args.block_id
    block_size = 500000
    start_id = block_id * block_size + 1
    end_id = (block_id + 1) * block_size

    compound_file_name = "Compound_{:09d}_{:09d}.sdf.gz".format(start_id, end_id)
    download_and_extract_compound_file(PubChemSTM_datasets_home_folder, compound_file_name)

## Model


In [ ]:
from torch import nn
from torch.nn import functional as F
from collections.abc import Sequence


class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dims, batch_norm=False, activation="relu", dropout=0):
        super(MLP, self).__init__()

        if not isinstance(hidden_dims, Sequence):
            hidden_dims = [hidden_dims]
        self.dims = [input_dim] + hidden_dims

        if isinstance(activation, str):
            self.activation = getattr(F, activation)
        else:
            self.activation = activation
        if dropout:
            self.dropout = nn.Dropout(dropout)
        else:
            self.dropout = None

        self.layers = nn.ModuleList()
        for i in range(len(self.dims) - 1):
            self.layers.append(nn.Linear(self.dims[i], self.dims[i + 1]))
        if batch_norm:
            self.batch_norms = nn.ModuleList()
            for i in range(len(self.dims) - 2):
                self.batch_norms.append(nn.BatchNorm1d(self.dims[i + 1]))
        else:
            self.batch_norms = None

    def forward(self, input):
        layer_input = input

        for i, layer in enumerate(self.layers):
            hidden = layer(layer_input)
            if i < len(self.layers) - 1:
                if self.batch_norms:
                    x = hidden.flatten(0, -2)
                    hidden = self.batch_norms[i](x).view_as(hidden)
                hidden = self.activation(hidden)
                if self.dropout:
                    hidden = self.dropout(hidden)
            if hidden.shape == layer_input.shape:
                hidden = hidden + layer_input
            layer_input = hidden

        return hidden

# Results

# Discussion

## Reproducability

Given the state of my initial results and the unexpected computational complexity of training, I do not believe that this paper will be reproducible within the timeline of this project. I even reached out to Shangchao Liu, the first author of the original paper, for guidance and while he indicated that he was "pretty sure" the results could be reproduced, he acknowledged that the setup, processing, and training would be a painful process. To that point, even after having successfully completed the initial setup, preprocessing, and beginning the pretraining process with an NVIDA RTX 4090 GPU running for the past 4 days, I'm still only on epoch 8/100; therefore, given these complexities, I do not believe that this project will be reproducible within the scope of this project.

## Setbacks and Diffuclties

The most major setback I've faced thus far in reproducing the results of this paper have been the significant amount of time sunk into the intial setup and preprocessing of the data. From a setup standpoint, I realized early on in my testing that many of the packages required for running this model are either deprecated or suffer from version incompatibilities. In total, I made 23 setup attempts with only the last succeeding. A significant amount of time has been invested in refactoring the source code, testing different version combinations for packages, using 3 different operating systems, and reaching out to the author for support. To finally get through the setup process, I had to take everything I had learned from my testing and apply it to a brand new computer I purchased exclusively to run Ubuntu, per the suggestion of the author. The several open issues on the GitHub repo containing the original source code, and the analysis of my peer groups trying to replicate this paper have confirmed my own experience that the setup for this project needs to be improved substantially to enable future researchers to replicate this work.

Another major setback I've experienced has been with regard to computational complexity. I assumed, naively, that my RTX 4090 card that I've used for training other models without issue would be suitable for this project. In reality, even with the top-of-the-line consumer GPU at my disposal, the pretraining process is taking an exhaustive and prohibitive amount of time. Though I've left the pretraining script running for several days at this point, it has only completed 8 of the 100 epochs necessary to complete. The authors make note of checkpoints with a pretrained model that can be used, and I considered doing this to cut down on time, but if I'm only using their pretrained model that seems to defeat the point of this project. Moreover, even if I continue to allow the pretraining script to run, I've estimated that it will take ~30 days to complete at its current pace, meaning even by the end of the semester, I would have close to nothing to show for all my efforts in attempting to reproduce this paper. To that end, as I will expound on the next section, I believe it would be warranted for me to switch my project topic and idenfity one of my backup papers that will be more feasible to reproduce for he purposes of this project.

## Future Plans

Given everything discussed in the previous sections regarding the difficulty of setup and the computational limitations that we're currently faced with, the best option seems to be requesting the ability to change our project to reproduce one of our backup papers instead. This is disappointing because on one hand, the research done by Liu et al. (2023) is truly groundbreaking and interesting from a deep learning perspective, but on the other hand, the lack of documentation for updated frameworks and the extensive need for computing has thrown a wrench into every expectation for this project. It would be in our best interest to switch papers as soon as possible and start again from scratch, as the current pace of the pretraining process does not bode well for our final project submission timeline.

# References

Liu, S., Nie, W., Wang, C., Lu, J., Qiao, Z., Liu, L., ... & Anandkumar, A. (2023). Multi-modal molecule structure–text model for text-based retrieval and editing. Nature Machine Intelligence, 5(12), 1447-1457.